<a href="https://colab.research.google.com/github/AyoubMelliti/AyoubMelliti/blob/main/legislatives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Premier tour

In [2]:
# -*- coding: utf-8 -*-
# Notebook : Analyse des évolutions des votes aux législatives (2012, 2017, 2022, 2024)

import pandas as pd
import requests
import io
import os

# Pour les visualisations
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

years = [2024]
years_urls = {}
years_urls[2024]='https://www.data.gouv.fr/api/1/datasets/r/41ed46cd-77c2-4ecc-b8eb-374aa953ca39'
# --- Fonctions utilitaires pour charger les données ---

def download_csv(url, sep=",", encoding="utf-8"):
    """
    Télécharge un fichier CSV depuis une URL et le charge dans un DataFrame pandas.
    """
    resp = requests.get(url)
    resp.raise_for_status()
    # On peut avoir du ; comme séparateur selon les fichiers
    return pd.read_csv(io.StringIO(resp.text), sep=sep, encoding=encoding)


def load_result(url):
    try:
      results = download_csv('url')
    except Exception as e:
      print(f"  Problème pour {y} : {e}")

    return



In [6]:
# --- Chargement des données pour les années désirées ---


data_general = {}
data_candidat = {}
for y in years:
    print(f"Chargement pour {y} ...")
    try:
        gen, cand = load_legislatives(y)
        data_general[y] = gen
        data_candidat[y] = cand
        print(f"  Général : {gen.shape}, Candidats : {cand.shape}")
    except Exception as e:
        print(f"  Erreur pour {y} : {e}")


Chargement pour 2024 ...
  Erreur pour 2024 : 404 Client Error: NOT FOUND for url: https://www.data.gouv.fr/fr/datasets/r/general-results.csv


In [1]:






# --- Harmonisation des noms de partis / nuançage ---

def harmonize_parti(df, col='nuance'):
    """
    Harmoniser les noms des partis ou nuances pour avoir une cohérence entre années
    (Exemple : 'RN' et 'Rassemblement National', 'PS', 'Parti Socialiste', etc.)
    """
    mapping = {
        'Rassemblement National': 'RN',
        'Front national': 'RN',
        'La République en marche': 'LREM',
        'Ensemble': 'ENS',
        'LR': 'LR',
        'Les Républicains': 'LR',
        'Parti socialiste': 'PS',
        'PS': 'PS',
        'La France insoumise': 'LFI',
        'EELV': 'EELV',
        'Europe Écologie Les Verts': 'EELV',
        # etc. Ajouter les autres partis ou nuancés que vous voulez suivre
    }
    df[col] = df[col].map(lambda x: mapping.get(x, x))
    return df

for y in years:
    dfc = data_candidat[y]
    if 'nuance' in dfc.columns:
        data_candidat[y] = harmonize_parti(dfc, col='nuance')
    elif 'parti' in dfc.columns:
        data_candidat[y] = harmonize_parti(dfc, col='parti')
    else:
        print(f"  Attention : pas de colonne nuance/parti pour {y}")


# --- Calcul de la part de voix d’un parti selon la circonscription, pour chaque année ---


def compute_share_by_circo(year, party, df_cand, df_gen):
    """
    Pour l'année donnée, calculer pour chaque circonscription la part de voix obtenue par le parti donné.
    Hypothèses :
      - df_cand contient une colonne 'nuance' ou 'parti' indiquant l'appartenance
      - df_cand contient une colonne 'voix' pour les voix du candidat
      - df_gen contient une colonne 'voix_exprimes' ou équivalent pour total exprimé
      - df_cand et df_gen ont une colonne 'circonscription' identifiant la circonscription législative
    Retourne un DataFrame avec colonisation circonscription, part du parti, total exprimé, voix du parti
    """
    if 'voix' not in df_cand.columns:
        raise ValueError("La colonne 'voix' n'existe pas dans df_cand")
    if 'voix_exprimes' in df_gen.columns:
        total_col = 'voix_exprimes'
    elif 'exprimés' in df_gen.columns:
        total_col = 'exprimés'
    else:
        # chercher dans les colonnes imprimées
        raise ValueError("Impossible de trouver la colonne du total des voix exprimées dans général")

    # nettoyer les colonnes de circonscription
    # selon que la colonne s’appelle 'circonscription', 'lib_circonscription' ou autre
    circ_col = 'circonscription' if 'circonscription' in df_cand.columns else None
    if circ_col is None:
        raise ValueError("Pas de colonne circonscription dans df_cand")

    # Votes du parti par circo
    dfp = df_cand[ df_cand['nuance'] == party ].groupby(circ_col)['voix'].sum().reset_index().rename(columns={'voix': 'voix_parti'})
    # Total exprimés par circo (depuis df_gen)
    dft = df_gen.groupby(circ_col)[total_col].sum().reset_index().rename(columns={total_col: 'voix_exprimes'})

    dfm = pd.merge(dfp, dft, on=circ_col, how='inner')
    dfm['part_parti'] = dfm['voix_parti'] / dfm['voix_exprimes']
    dfm['année'] = year
    return dfm


# --- Exemple d’analyse : évolution de la part du RN (ou autre) dans quelques circonscriptions ---


party_of_interest = "RN"  # modifier selon besoin
share_dfs = []

for y in years:
    try:
        df_share = compute_share_by_circo(y, party_of_interest, data_candidat[y], data_general[y])
        share_dfs.append(df_share)
    except Exception as e:
        print(f"  Problème pour {y} : {e}")

df_all = pd.concat(share_dfs, ignore_index=True)


# --- Visualisation : tracé de l’évolution de la part de voix par circonscription pour ce parti ---

plt.figure(figsize=(14, 7))
sns.lineplot(data=df_all, x='année', y='part_parti', hue='circonscription', legend=False, alpha=0.3)
plt.title(f"Évolution de la part de voix de {party_of_interest} par circonscription (2012-2024)")
plt.ylabel("Part de voix exprimées")
plt.xlabel("Année")
plt.show()


# --- Visualisation : évolution moyenne nationale du parti ---

df_national = df_all.groupby('année')['part_parti'].mean().reset_index()

plt.figure(figsize=(8,5))
sns.barplot(data=df_national, x='année', y='part_parti', palette="viridis")
plt.title(f"Part moyenne de voix de {party_of_interest} (toutes circonscriptions confondues)")
plt.ylabel("Part moyenne (circonscription)")
plt.xlabel("Année")
plt.show()




Chargement pour 2012 ...
  Erreur pour 2012 : 404 Client Error: NOT FOUND for url: https://www.data.gouv.fr/fr/datasets/r/general-results.csv
Chargement pour 2017 ...
  Erreur pour 2017 : 404 Client Error: NOT FOUND for url: https://www.data.gouv.fr/fr/datasets/r/general-results.csv
Chargement pour 2022 ...
  Erreur pour 2022 : 404 Client Error: NOT FOUND for url: https://www.data.gouv.fr/fr/datasets/r/general-results.csv
Chargement pour 2024 ...
  Erreur pour 2024 : 404 Client Error: NOT FOUND for url: https://www.data.gouv.fr/fr/datasets/r/general-results.csv


KeyError: 2012